# 課題 1.1　線形回帰分析

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

autos = pd.read_csv('data/imports-85.data', na_values='?')
autos.columns = ['symboling', 'normalized_losses', 'make', 'fuel_type', 'aspiration',
                    'num_of_doors', 'body_style', 'drive_wheels', 'engine_location',
                    'wheel_base', 'length','width', 'height', 'curb_weight',
                    'engine_type', 'num_of_cylinders', 'engine_size', 'fuel_system',
                    'bore', 'stroke', 'compression_ratio', 'horsepower', 'peak_rpm',
                    'city_mpg', 'highway_mpg','price']
# 表示する最大列数の設定
pd.set_option('display.max_columns', len(autos.columns))
# 欠損値 NaN がある行の除去
autos = autos.dropna()
# 確認
autos

In [ ]:
# price 以外の量的データのラベルを昇順にソート
qd_labels = autos.describe().columns.drop('price').sort_values()
# 確認
qd_labels

<hr>

### 1.1.1 VIFが5以上の説明変数を除外

price 以外の量的データである説明変数について、昇順にソートしたラベル qd_labels の先頭からVIFが5以上の変数を除外し、残った説明変数について、説明変数とVIFを行とする pandas DataFrame を作成し、表示しなさい。

### 解釈2

＜説明変数、目的変数＞のVIFが5以上であれば、この目的変数は次で用いる説明変数から除去する。

1.
$x_{1}$
を目的変数、
$x_{2} ... x_{n}$
を説明変数した時のVIFが5以上ならば
$x_{1}$
は除去。

2.
次は
$x_{2}$
を目的変数、
$x_{3} ... x_{n}$
を説明変数した時のVIFを計算。

- city_mpg, highway_mpg は高い相関を持つため、先に現れる city_mpg は highway_mpg によって説明されるため、除去される。しかしながら、city_mpg が除去された残りの変数からは、highway_mpg は説明されないため、最終的に説明変数として残る。

In [ ]:
model = LinearRegression()

def LR_score(df, Y_label):
    # 説明変数、目的変数
    X = df.drop([Y_label], axis=1)
    Y = df[Y_label]
    # 学習、予測値
    model.fit(X, Y)
    Y_predict = model.predict(X)
    # 決定係数
    return r2_score(Y, Y_predict)

# VIFを計算する関数を定義
# - df: 対象とする全ての変数が入った DataFrame
# - df_vif: VIF を入れる DataFrame
def drop_vif(df, df_vif):
    dropped = False

    # df.columns（全ての変数ラベル）から、順番に1つずつ取り出す
    for Y_label in df.columns:
        # VIF の計算
        _vif = 1 / (1 - LR_score(df, Y_label))
        # DataFrame への代入
        df_vif.loc[Y_label] = _vif
        # VIF が 5 以上ならば除外
        if _vif >= 5:
            # データセットから除外
            df = df.drop([Y_label], axis=1)
            # VIF を入れる DataFrame からも除外
            df_vif = df_vif.drop([Y_label])
            # 確認
            print(Y_label, _vif)
            # 除外が発生
            dropped = True

    # df: VIF が 5 以上の変数を除外した DataFrame
    # df_vif: VIF が 5 以上の変数を除外した VIF の DataFrame
    # dropped: 除外の有無を示すフラグ
    return df, df_vif, dropped

In [ ]:
# 量的データ
df = autos[qd_labels]

# VIF を入れる DataFrame の初期化
df_vif = pd.DataFrame(index=qd_labels, columns=['vif'])

# 除外がなくなるまで繰り返す
_c = True
while (_c):
    df, df_vif, _c = drop_vif(df, df_vif)

# 確認
df_vif

<hr>

### 1.1.2 決定係数

すべての質的データの説明変数と1.1.1で残った説明変数を合わせて説明変数、price を目的変数として重回帰モデルを作成し、そのモデルの決定係数を求めなさい。<br>
小数点以下4桁目を四捨五入し、小数点以下3桁まで答えること（0.7625ならば0.763が答え）。

In [ ]:
# qd_labels の列をdropして質的データだけを残し、df_vif に残った説明変数を追加した列ラベルを作成
X_labels = autos.columns.drop(qd_labels).append(df_vif.index)
df = autos[X_labels]
df = pd.get_dummies(data=df, drop_first=True)

R2 = LR_score(df, 'price')
print('R2={:.3f}'.format(R2))

<hr>

### 1.1.3 回帰係数

1.1.2で作成したモデルの回帰係数を、説明変数と回帰係数を行とする pandas DataFrame を作成し、降順で表示しなさい（行数が多い場合、途中の表示が省略されないようにすること）。

In [ ]:
# 回帰係数
pd.set_option('display.max_columns', len(df.columns))
pd.DataFrame(model.coef_, index=df.columns.drop('price'), columns=['coefficient']).\
    sort_values(by='coefficient', ascending=False)